Estatística - Teste de Hipóteses

Considerando a base de dados experimento_test_ab.csv responda as
questões abaixo:


1. Qual dos cenários tem a maior taxa de conversão?
 


In [17]:
import pandas as pd
from scipy import stats
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

In [18]:
experimento = "experimento_teste_ab.csv"

df_experimento = pd.read_csv(experimento)

df_experimento.head()

,Visitante_ID,Versão_Página,Visualizações,Conversões
0,1,A,167,1
1,2,A,160,0
2,3,A,170,0
3,4,A,161,1
4,5,A,104,0


In [20]:

# Agrupando por versão da página e calculando taxa de conversão
taxa_conversao = df_experimento.groupby("Versão_Página")["Conversões"].agg(["sum", "count"])
taxa_conversao["Taxa_Conversao"] = taxa_conversao["sum"] / taxa_conversao["count"]

print(taxa_conversao)

               sum  count  Taxa_Conversao
Versão_Página                            
A              105   1000           0.105
B              125   1000           0.125


2. Calcule qual o tamanho da amostra necessária para o desenvolvimento
de um teste A/B, seguindo os seguintes critérios:
a. O cenário A, da base, como o inicial, que funciona hoje.
b. Considere 95% de confiança de que o efeito na conversão não foi
aleatório com um nível de significância de 5% (alpha = 0,05).
c. Também considere 80% de certeza conseguir capturar o efeito
da nova abordagem.
d. O aumento para 10% de conversão.



In [21]:


# 1. Taxa de conversão atual (grupo A)
conversao_A = df_experimento[df_experimento["Versão_Página"] == "A"]["Conversões"].sum()
n_A = df_experimento[df_experimento["Versão_Página"] == "A"].shape[0]
p1 = conversao_A / n_A

# 2. Conversão esperada para o novo grupo B
p2 = 0.10  # 10%

# 3. Tamanho do efeito (effect size de Cohen)
effect_size = proportion_effectsize(p1, p2)

# 4. Calculando o tamanho da amostra
analysis = NormalIndPower()
amostra_necessaria = analysis.solve_power(effect_size=effect_size, power=0.8, alpha=0.05, ratio=1)

print(f"Tamanho mínimo da amostra por grupo: {round(amostra_necessaria)}")


Tamanho mínimo da amostra por grupo: 57756


Verifique a base pacientes.csv, os dados são fictícios. A sintetiza uma base
de dados de um hospital que trata de pacientes com problemas cardíacos.
Considere a base como a população, portanto as estatísticas da população
são conhecidas. Responda as perguntas abaixo:
3. Considerando uma amostra de 45 números que representam o index do
dataframe, índices= ([909, 751, 402, 400, 726, 39, 184, 269, 255, 769,
209, 715, 677, 381, 793, 697, 89, 280, 232, 756, 358, 36, 439, 768, 967,
699, 473, 222, 89, 639, 883, 558, 757, 84, 907, 895, 217, 224, 311, 348,
146, 505, 273, 957, 362]). Considerando essa amostra é possível dizer
que a idade média das pessoas com problemas cardíacos é maior que
50 anos? Nível de significância igual a 5%.


In [22]:
pacientes = "pacientes.csv"

df_pacientes = pd.read_csv(pacientes)

df_pacientes

,Unnamed: 0,Pressao_Arterial,Idade,Genero,Etnia,Estado_Saude,Nome_Genero,Nome_Etnia,Nome_Estado_Saude
0,0,142.755809,31,1,1,1,Feminino,Afro-americano,Com condições de saúde adicionais
1,1,161.878468,35,1,1,0,Feminino,Afro-americano,Saudável
2,2,161.508259,39,0,3,1,Masculino,Asiático,Com condições de saúde adicionais
3,3,144.979408,43,1,3,1,Feminino,Asiático,Com condições de saúde adicionais
4,4,162.502954,55,0,2,1,Masculino,Hispânico,Com condições de saúde adicionais
...,...,...,...,...,...,...,...,...,...
995,995,136.381404,58,1,0,1,Feminino,Caucasiano,Com condições de saúde adicionais
996,996,161.801012,41,0,3,1,Masculino,Asiático,Com condições de saúde adicionais
997,997,134.390624,70,0,0,0,Masculino,Caucasiano,Saudável
998,998,160.818609,33,1,0,0,Feminino,Caucasiano,Saudável


In [41]:

# Índices da amostra
indices = [909, 751, 402, 400, 726, 39, 184, 269, 255, 769,
           209, 715, 677, 381, 793, 697, 89, 280, 232, 756,
           358, 36, 439, 768, 967, 699, 473, 222, 89, 639,
           883, 558, 757, 84, 907, 895, 217, 224, 311, 348,
           146, 505, 273, 957, 362]

# Criar a amostra
amostra = df_pacientes.loc[indices]

# Verificar a coluna de idade (supondo que o nome é 'idade')
media_amostra = amostra["Idade"].mean()
desvio_amostral = amostra["Idade"].std(ddof=1)
n = len(amostra)

# Teste t de uma amostra contra H0: mu <= 50
t_stat, p_valor_bilateral = stats.ttest_1samp(amostra["Idade"], popmean=50)

# Como o teste é unilateral (maior que 50), ajusta o p-valor
p_valor_unilateral = p_valor_bilateral / 2

# Mostrar resultados
print(f"Média da amostra: {media_amostra:.2f}")
print(f"T-statística: {t_stat:.4f}")
print(f"P-valor (unilateral): {p_valor_unilateral:.4f}")

# Decisão
if (t_stat > 0) and (p_valor_unilateral < 0.05):
    print(" Rejeitamos H0: A idade média é maior que 50 anos.")
else:
    print(" Não há evidência suficiente para afirmar que a média é maior que 50 anos.")


Média da amostra: 50.09
T-statística: 0.0523
P-valor (unilateral): 0.4793
 Não há evidência suficiente para afirmar que a média é maior que 50 anos.


4. Queremos entender que tipo de amostra estamos lidando se dividirmos
os conjuntos em 2, sendo um com pessoas que têm condições de
saúde adicionais e o outro com pessoas saudáveis. Seria dependente
ou independente?


In [40]:

# Verificar os valores únicos na coluna "Estado_Saude"
print("Valores únicos na coluna 'Estado_Saude':")
print(df_pacientes["Estado_Saude"].unique())

# Criar os dois grupos
grupo_com_condicao = df_pacientes[df_pacientes["Estado_Saude"] == "Com condições de saúde adicionais"]
grupo_saudavel = df_pacientes[df_pacientes["Estado_Saude"] == "Saudável"]

# Verificar se há interseção entre os dois grupos (pacientes repetidos)
interseccao = grupo_com_condicao.index.intersection(grupo_saudavel.index)

# Resultado
if len(interseccao) == 0:
    print("Os grupos são independentes: não há pacientes em comum.")
else:
    print("Os grupos são dependentes: há pacientes em ambos os grupos.")
    print("Pacientes repetidos (índices):", interseccao.tolist())


Valores únicos na coluna 'Estado_Saude':
[1 0]
Os grupos são independentes: não há pacientes em comum.


5. Agora considere o um conjunto de pessoas aleatória que representam
o index do dataframe, índices = ([690, 894, 67, 201, 364, 19, 60, 319,
588, 643, 855, 623, 530, 174, 105, 693, 6, 462, 973, 607, 811, 346, 354,
966, 943, 372]), podemos dizer que a pressão arterial média para
pacientes com condições de saúde adicionais é igual à pressão arterial
média para pacientes sem condições adicionais de saúde? Considere o
nível de significância a 6%.

In [39]:

# Índices da amostra fornecida
indices = [690, 894, 67, 201, 364, 19, 60, 319, 588, 643, 855, 623, 530, 174, 105, 
           693, 6, 462, 973, 607, 811, 346, 354, 966, 943, 372]

# Criar a amostra usando os índices
amostra = df_pacientes.loc[indices]

# Dividir a amostra em dois grupos: com e sem condições de saúde adicionais
grupo_com_condicao = amostra[amostra["Estado_Saude"] == "Com condições de saúde adicionais"]
grupo_saudavel = amostra[amostra["Estado_Saude"] == "Saudável"]

# Verificar as médias das pressões arteriais
media_com_condicao = grupo_com_condicao["Pressao_Arterial"].mean()
media_saudavel = grupo_saudavel["Pressao_Arterial"].mean()

print(f"Média pressão arterial (com condições de saúde adicionais): {media_com_condicao:.2f}")
print(f"Média pressão arterial (saudáveis): {media_saudavel:.2f}")

# Realizar o teste t para duas amostras independentes
t_stat, p_valor = stats.ttest_ind(grupo_com_condicao["Pressao_Arterial"], grupo_saudavel["Pressao_Arterial"])

# Resultado do teste (teste bilateral, então p-valor é o mesmo para comparar as médias)
print(f"T-statística: {t_stat:.4f}")
print(f"P-valor: {p_valor:.4f}")

# Decisão (nível de significância 6% = 0,06)
if p_valor < 0.06:
    print("Rejeitamos H0: As médias das pressões arteriais são diferentes.")
else:
    print("Não rejeitamos H0: Não há evidências suficientes para afirmar que as médias são diferentes.")


Média pressão arterial (com condições de saúde adicionais): nan
Média pressão arterial (saudáveis): nan
T-statística: nan
P-valor: nan
Não rejeitamos H0: Não há evidências suficientes para afirmar que as médias são diferentes.


C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\_lib\deprecation.py:234: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return f(*args, **kwargs)


6. Existe uma diferença significativa na pressão arterial média entre
diferentes grupos étnicos nesta população? (Teste ANOVA, alpha é 5%)
a. Hipótese Nula (H0): A pressão arterial média é a mesma em todos
os grupos étnicos.
b. Hipótese Alternativa (H1): Há uma diferença significativa na
pressão arterial média entre pelo menos dois grupos étnicos
nesta população.



In [38]:

# Agrupar os dados por etnia
grupos_etnicos = df_pacientes.groupby("Nome_Etnia")["Pressao_Arterial"].apply(list)

# Realizar o teste ANOVA
f_stat, p_valor = stats.f_oneway(*grupos_etnicos)

# Resultado do teste
print(f"F-statística: {f_stat:.4f}")
print(f"P-valor: {p_valor:.4f}")

# Decisão com base no p-valor
if p_valor < 0.05:
    print("Rejeitamos H0: Existe uma diferença significativa na pressão arterial média entre os grupos étnicos.")
else:
    print("Não rejeitamos H0: Não há evidências suficientes para afirmar que as médias das pressões arteriais são diferentes entre os grupos étnicos.")


F-statística: 0.2325
P-valor: 0.8738
Não rejeitamos H0: Não há evidências suficientes para afirmar que as médias das pressões arteriais são diferentes entre os grupos étnicos.


7. Lúcia é uma pesquisadora e tem o objetivo de entender a relação de
gênero neste grupo de pacientes. Acredita-se que há uma relação
entre o sexo e condições de saúde adicionais. (Teste qui-quadrado)


In [37]:

# Criar a tabela de contingência entre Gênero e Estado de Saúde
tabela_contingencia = pd.crosstab(df_pacientes["Nome_Genero"], df_pacientes["Nome_Estado_Saude"])

# Realizar o teste Qui-quadrado
chi2_stat, p_valor, dof, expected = stats.chi2_contingency(tabela_contingencia)

# Resultado do teste
print(f"Estatística Qui-quadrado: {chi2_stat:.4f}")
print(f"P-valor: {p_valor:.4f}")
print(f"Graus de liberdade: {dof}")
print(f"Valores esperados: \n{expected}")

# Decisão com base no p-valor
if p_valor < 0.05:
    print("Rejeitamos H0: Existe uma relação significativa entre o gênero e as condições de saúde adicionais.")
else:
    print("Não rejeitamos H0: Não há evidências suficientes para afirmar que o gênero e as condições de saúde adicionais estão relacionados.")


Estatística Qui-quadrado: 0.0000
P-valor: 1.0000
Graus de liberdade: 1
Valores esperados: 
[[256.53 246.47]
 [253.47 243.53]]
Não rejeitamos H0: Não há evidências suficientes para afirmar que o gênero e as condições de saúde adicionais estão relacionados.


8. Existe uma associação entre a idade dos pacientes e sua pressão
arterial?
a. Hipótese Nula (H0): se a pressão arterial é independente da idade
b. Hipótese Alternativa (H1): a pressão arterial esta associada a
idade



In [42]:

# Realizar o teste de correlação de Pearson entre Idade e Pressão Arterial
correlacao, p_valor = stats.pearsonr(df_pacientes["Idade"], df_pacientes["Pressao_Arterial"])

# Resultado do teste
print(f"Coeficiente de Correlação de Pearson: {correlacao:.4f}")
print(f"P-valor: {p_valor:.4f}")

# Decisão com base no p-valor
if p_valor < 0.05:
    print("Rejeitamos H0: Existe uma associação significativa entre a idade e a pressão arterial.")
else:
    print("Não rejeitamos H0: Não há evidências suficientes para afirmar que a pressão arterial está associada à idade.")


Coeficiente de Correlação de Pearson: -0.0214
P-valor: 0.4994
Não rejeitamos H0: Não há evidências suficientes para afirmar que a pressão arterial está associada à idade.


9. Qual é o intervalo de confiança para a média da pressão arterial entre
os pacientes com condições de saúde adicionais? (nível de confiança
95%)


In [43]:

# Filtrar os pacientes com condições de saúde adicionais
pacientes_condicoes_adicionais = df_pacientes[df_pacientes["Nome_Estado_Saude"] == "Com condições de saúde adicionais"]

# Calcular a média e o desvio padrão da pressão arterial para este grupo
media_pressao = pacientes_condicoes_adicionais["Pressao_Arterial"].mean()
desvio_padrao = pacientes_condicoes_adicionais["Pressao_Arterial"].std()
n = len(pacientes_condicoes_adicionais)

# Definir o nível de confiança de 95%
nivel_confianca = 0.95
alpha = 1 - nivel_confianca

# Calcular o t-valor para o nível de confiança de 95%
t_valor = stats.t.ppf(1 - alpha/2, df=n-1)

# Calcular o erro padrão
erro_padrao = desvio_padrao / (n**0.5)

# Calcular o intervalo de confiança
intervalo_inf = media_pressao - t_valor * erro_padrao
intervalo_sup = media_pressao + t_valor * erro_padrao

# Exibir o intervalo de confiança
print(f"Intervalo de confiança de 95% para a média da pressão arterial: ({intervalo_inf:.2f}, {intervalo_sup:.2f})")


Intervalo de confiança de 95% para a média da pressão arterial: (149.22, 151.02)


10. A distribuição da pressão arterial na população segue uma distribuição
normal?
a. Hipótese Nula (H0): A distribuição da pressão arterial na
população segue uma distribuição normal.
b. Hipótese Alternativa (H1): A distribuição da pressão arterial na
população não segue uma distribuição normal.

In [44]:

# Realizar o teste de normalidade de Shapiro-Wilk na pressão arterial
estatistica, p_valor = stats.shapiro(df_pacientes["Pressao_Arterial"])

# Exibir o resultado do teste
print(f"Estatística do teste: {estatistica:.4f}")
print(f"P-valor: {p_valor:.4f}")

# Decisão com base no p-valor
if p_valor < 0.05:
    print("Rejeitamos H0: A distribuição da pressão arterial não segue uma distribuição normal.")
else:
    print("Não rejeitamos H0: A distribuição da pressão arterial segue uma distribuição normal.")

Estatística do teste: 0.9970
P-valor: 0.0607
Não rejeitamos H0: A distribuição da pressão arterial segue uma distribuição normal.
